# CHF Shelter Data Federated Learning Demo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit


from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
tqdm.pandas()

Agencies: 4, 13, 55, 188, 213, 225, 330, 333

In [ ]:
LDays = 548
Data_Days = 90
Data_periods = 10
Agency = 333

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
tbl = pd.read_parquet('MergedShelterData-Nov17.parquet')

In [ ]:
tbl

## EDA
---

In [ ]:
print(f'Dates: {tbl.Date.min()} to {tbl.Date.max()}')

In [ ]:
nPrsn = len(tbl.ClientId.unique())
print(f'{nPrsn} people in the data.')

In [ ]:
print(f'{len(tbl.Agency.unique())} different shelters.')

#### Number of people who use different shelters.

Total Population

In [ ]:
nShelter = tbl.groupby('ClientId').progress_apply(lambda x: len(x.Agency.unique()))

In [ ]:
def number_of_shelter_breakdown(nShelter,nPrsn):
    hist = nShelter.value_counts()
    for nS in hist.index.sort_values():
        print(f'{hist[nS]}/{nPrsn} ({100*hist[nS]/nPrsn:.2f}%) people used {nS} shelters.')        

In [ ]:
number_of_shelter_breakdown(nShelter,nPrsn)

Heavy System Users

In [ ]:
nStay = tbl.groupby('ClientId').Date.count()

In [ ]:
heavyPctl = 0.95
heavyIds = nStay.sort_values().iloc[int(nPrsn*heavyPctl):].index.to_numpy()

In [ ]:
number_of_shelter_breakdown(nShelter[heavyIds],len(heavyIds))

In [ ]:
agency_dfs = {}
unique_agencies = tbl['Agency'].unique()

for agency in unique_agencies:
    agency_dfs[agency] = tbl[tbl['Agency'] == agency]


df_agency = agency_dfs[Agency] 


In [ ]:
tbl = df_agency
# Assuming tbl is your dataframe and 'ClientId' is the column you want to modify
tbl['ClientId'] = tbl['ClientId'].astype(str) + f'_{Agency}'

## Labelling
---
- We'll use cluster based analysis on total stay and total number of stay episode values to label our data set.
- More information on this methodology here: https://arxiv.org/abs/2210.13619

In [ ]:
# Assuming tbl is your DataFrame
tbl['Date'] = pd.to_datetime(tbl['Date'])

def filter_group(group):
    group = group.sort_values('Date')
    min_date = group['Date'].min()
    # Keep only dates within the first 90 days
    group = group[group['Date'] <= min_date + pd.Timedelta(days=LDays)]
    # Drop duplicates in the Date column
    group = group.drop_duplicates(subset='Date', keep=False)
    return group

tbl = tbl.groupby('ClientId').apply(filter_group).reset_index(drop=True)

In [ ]:
episodeGap = 30 # days

def calc_stays_and_gaps(tbl):    
    stayDates = tbl.Date.drop_duplicates().sort_values() 
    nStay = len(stayDates)

    gapVals = stayDates.diff()
    nEpi = len(gapVals.loc[gapVals >= pd.Timedelta(f'{episodeGap} day') ])+1
    
    return pd.Series({ 'NStays': nStay, 'NEpisodes': nEpi })

In [ ]:
tblStayGap = tbl.groupby('ClientId').progress_apply(calc_stays_and_gaps)

## Labelling at Individual Shelters
---
- Assume the centralized labels are the ground truth and compare how close labels generated in each shelter get.

In [ ]:
nShelters = tbl['Agency'].drop_duplicates()

## Chronic Shelter Use Prediction at Individual Shelters
---

Step 1: Find the total time frame.



Step 2: Implement Windowing for like 4 months.

In [ ]:
import pandas as pd

# Assuming tbl is your DataFrame
tbl['Date'] = pd.to_datetime(tbl['Date'])

def filter_group(group):
    group = group.sort_values('Date')
    min_date = group['Date'].min()
    # Keep only dates within the first 90 days
    group = group[group['Date'] <= min_date + pd.Timedelta(days=Data_Days)]
    # Drop duplicates in the Date column
    group = group.drop_duplicates(subset='Date', keep=False)
    return group

filtered_tbl = tbl.groupby('ClientId').apply(filter_group).reset_index(drop=True)


In [ ]:
episodeGap = 30 # days

def calc_stays_and_gaps(tbl):    
    stayDates = tbl.Date.drop_duplicates().sort_values() 
    nStay = len(stayDates)

    gapVals = stayDates.diff()
    nEpi = len(gapVals.loc[gapVals >= pd.Timedelta(f'{episodeGap} day') ])+1
    
    return pd.Series({ 'NStays': nStay, 'NEpisodes': nEpi })

In [ ]:
filtered_tbl_2 = filtered_tbl.groupby('ClientId').progress_apply(calc_stays_and_gaps)

In [ ]:
filtered_tbl['Sleep'] = 1

In [ ]:
import pandas as pd

def aggregate_to_30_day_windows(group):
    group.set_index('Date', inplace=True)
    # Initialize an empty DataFrame to store the final result for this group
    final_group = pd.DataFrame()
    calculated_freq = f'{int(Data_Days/Data_periods)}D'
    # Create periods of 30 days each, starting from the minimum date
    for period_start in pd.date_range(start=group.index.min(), periods=Data_periods, freq=calculated_freq):
        # Define the end of the period (30 days after the start)
        period_end = period_start + pd.Timedelta(days=Data_Days/Data_periods)
        # Filter the group for events within the current 30-day period
        events_in_period = group[(group.index >= period_start) & (group.index < period_end)]
        # Count the number of events in the period
        event_count = len(events_in_period)
        # Add a row with the count of events and the correct ClientId
        count_row = pd.DataFrame({'Sleep': [event_count]}, index=[period_start])
        final_group = final_group.append(count_row)
    final_group['ClientId'] = group.name  # Add the ClientId to each row
    return final_group

# Assuming filtered_tbl is your pre-filtered DataFrame
aggregated_tbl = filtered_tbl.groupby('ClientId').apply(aggregate_to_30_day_windows).reset_index(drop=True)



This is where I assign shelters to the clients

In [ ]:
# This will create a series from 0, 1, 2, 0, 1, 2, etc.
helper_column = (aggregated_tbl.groupby('ClientId').cumcount() + 1).astype(str)

# Add this as a new column to the DataFrame
aggregated_tbl['CountId'] = 'Sleep_' + helper_column

# Pivot the table
pivot_table = aggregated_tbl.pivot(index='ClientId', columns='CountId', values='Sleep')

# Reset the index to turn ClientId into a column
pivot_table.reset_index(inplace=True)

unique_clients = filtered_tbl.drop_duplicates(subset='ClientId', keep='first')

# Set ClientId as index for easy lookup
pivot_table.set_index('ClientId', inplace=True)
unique_clients.set_index('ClientId', inplace=True)

# Update Agency in agg_tbl based on unique_clients
pivot_table['Agency'] = pivot_table.index.map(unique_clients['Agency'])

# Reset index if you want ClientId as a column
pivot_table.reset_index(inplace=True)


Combine Shelters over ClientID

Run a model in Pytorch

In [ ]:
pivot_table = pivot_table.fillna(0)

In [ ]:
unique_clients.reset_index(inplace=True)

In [ ]:
filtered_tbl_2.reset_index(inplace=True)

In [ ]:
pivot_table['Episodes'] = filtered_tbl_2['NEpisodes']
pivot_table['Stays'] = filtered_tbl_2['NStays']
pivot_table['Agency'] = unique_clients['Agency']

In [ ]:
# # Save aggregated_tbl as a CSV file
file_name = f'CHF_Data_1/CHF_{Agency}A_{Data_Days}D_{Data_periods}W.csv'
pivot_table.to_csv(file_name, index=False)